
<h1>Capstone Project - The Battle of Neighborhoods(Week1)</h1>

<h2>Selecting the best location to open an SUSHI BAR IN Manhattan, New York</h2>


<h3>Introduction/Business Problem</h3>

<p>The City of New York is famous for its excelllent cuisine. It's food culture includes an array of international cuisines influenced by the city's immigrant history. 

Sushi restaurants have become so popular in the United States now it seems that there is  one on every corner, not only in major cities but also in smaller cities. Starting a sushi restaurant can be a great business opportunity, but you need to distinguish yourself from others to enjoy long-term success.


<h3>Business Problem</h3>
<p>My client wants to open his business in Manhattan area, so I focus on that borough during my analysis. We define potential neighborhood based on the number of sushi bars which are operating right in each neighborhood. Manhattan has full potential but also is a very challenging district to open a business because of high competition. New sushi bar should be open in an area that inadequate neighborhood in this way the bar can attract more customers. Therefore, this analysis necessary to ensure that we have enough customers and that we are not so close to other sushi places.


<h3>Data Selection</h3>
<p>To identify the characteristics of our competitors' venues in Manhattan, we would first need to find out the number of sushi bars in Manhattan currently and their location.
We then used Google Map API to find their geographic coordinates based on their postal code addresses.
In Manhattan, there is 1763 sushi bars are currently operating. <br>
Next, we also used Google Map API to find their geographic coordinates of the 5 locations shortlisted for our sushi bar:
![image.png](attachment:image.png)

<h3>Methodology</h3>
<ul>
<li>Addresses are converted into their equivalent latitude and longitude values. 
<li>Foursquare API is used to explore neighborhoods in Manhattan, New York. 
<li>After that, explore function to get sushi restaurant categories in each neighborhood.
<li>Then using this feature to group the neighborhoods into clusters K-means clustering algorithm will be use to complete this task. And also, the Folium library to visualize the neighborhoods in Manhattan and its emerging clusters.
![image.png](attachment:image.png)
</ul>



<h3>Result</h3>
<p>Based on dataframe analysis above Cluster 3 (Upper West Side ) and Cluster 4 (Morningside Heights) areas are the best places to open a new sushi bar business.


<h3>Discussion</h3>
<ul>
<li>This analysis is performed on limited data. This may be right or may be wrong. But if good amount of data is available there is scope to come up with better results.
<li>There is high competition in Midtown and Soho so it is very risky to open business in these areas.
<li>Central Harlem has also potential where closes to Morningside Heights area.
<li>It can be done more detailed analysis by adding other factors such as transportation, demographics of inhabitants
</ul>

<h3>Conclusion</h3>
<p>Although all of the goals of this project were met there is definitely room for further improvement and development as noted below. However, the goals of the project were met and, with some more work, could easily be devleoped into a fully phledged application that could support the opening a business idea in an unknown location.


In [38]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


<h1>Download and Explore Dataset</h1>


<p>Download and Explore Dataset Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the files and placed it on the server, so you can simply run a wget command and access the data. So let's go ahead and do that.

In [11]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


In [12]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)


In [13]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [14]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [15]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [16]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [17]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [18]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'ZMHWBS0SR12Z3YDYVHJVTZPRK3U1ZP3I2TYQAJ5CU3JUHMB5'
CLIENT_SECRET = 'H3TT0XT3P5TIAFCV1Y2UUVLF42N44DICNKLUELK34H2TKLFR'
VERSION = '20181020'

In [19]:
#https://developer.foursquare.com/docs/resources/categories
#Sushi = 4bf58dd8d48988d1d2941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_sushi = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1d2941735')
newyork_venues_sushi.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Planet Tokyo,40.886233,-73.909479,Sushi Restaurant
1,Chinatown,40.715618,-73.994279,Sushi Hatsune,40.715994,-73.992859,Sushi Restaurant
2,Chinatown,40.715618,-73.994279,Shinsen,40.715608,-73.996611,Japanese Restaurant
3,Chinatown,40.715618,-73.994279,Nakaji,40.715912,-73.996597,Sushi Restaurant
4,Chinatown,40.715618,-73.994279,Mikaku Sushi,40.721419,-73.996731,Sushi Restaurant


In [20]:
newyork_venues_sushi.shape

(1097, 7)

In [21]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [22]:
map_newyork_sushi = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_sushi, 'red', map_newyork_sushi)

map_newyork_sushi

In [23]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [24]:
manhattan_grouped = newyork_venues_sushi.groupby('Neighborhood').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_sushi['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,22,22,22,22,22,22
Carnegie Hill,24,24,24,24,24,24
Central Harlem,3,3,3,3,3,3
Chelsea,42,42,42,42,42,42
Chinatown,24,24,24,24,24,24
Civic Center,31,31,31,31,31,31
Clinton,37,37,37,37,37,37
East Harlem,3,3,3,3,3,3
East Village,50,50,50,50,50,50


In [25]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_sushi[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_sushi['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()


,Neighborhood,Asian Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Noodle House,Ramen Restaurant,Restaurant,Sake Bar,Sandwich Place,Seafood Restaurant,Smoothie Shop,Steakhouse,Sushi Restaurant,Vegetarian / Vegan Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,Chinatown,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [26]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Asian Restaurant,Bakery,Bubble Tea Shop,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Noodle House,Ramen Restaurant,Restaurant,Sake Bar,Sandwich Place,Seafood Restaurant,Smoothie Shop,Steakhouse,Sushi Restaurant,Vegetarian / Vegan Restaurant
0,Battery Park City,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.045455,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.863636,0.000000
1,Carnegie Hill,0.041667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.125000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.750000,0.041667
2,Central Harlem,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,1.000000,0.000000
3,Chelsea,0.047619,0.00,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.095238,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.809524,0.023810
4,Chinatown,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.875000,0.000000
5,Civic Center,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.064516,0.032258,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.903226,0.000000
6,Clinton,0.027027,0.00,0.000000,0.027027,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.081081,0.000000,0.00,0.00,0.000000,0.00,0.027027,0.000000,0.000000,0.810811,0.000000
7,East Harlem,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,1.000000,0.000000
8,East Village,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.100000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.860000,0.020000
9,Financial District,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.181818,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.818182,0.000000


In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Sushi Restaurant,Japanese Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
1,Carnegie Hill,Sushi Restaurant,Japanese Restaurant,Indian Chinese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Restaurant,Bubble Tea Shop,Steakhouse,Smoothie Shop,Seafood Restaurant
2,Central Harlem,Sushi Restaurant,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
3,Chelsea,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Restaurant,Bubble Tea Shop,Steakhouse,Smoothie Shop,Seafood Restaurant
4,Chinatown,Sushi Restaurant,Japanese Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant


In [29]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([3, 1, 0, 1, 3, 3, 1, 0, 3, 4], dtype=int32)

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Sushi Restaurant,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,3,Sushi Restaurant,Japanese Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Sushi Restaurant,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,0,Sushi Restaurant,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Sushi Restaurant,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant


In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [32]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sushi Restaurant,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
2,Washington Heights,Sushi Restaurant,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
3,Inwood,Sushi Restaurant,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
4,Hamilton Heights,Sushi Restaurant,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
6,Central Harlem,Sushi Restaurant,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
7,East Harlem,Sushi Restaurant,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
18,Greenwich Village,Sushi Restaurant,Japanese Restaurant,Sake Bar,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Restaurant,Ramen Restaurant
21,Tribeca,Sushi Restaurant,Noodle House,Asian Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant


In [33]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Restaurant,Bubble Tea Shop,Steakhouse,Smoothie Shop,Seafood Restaurant
9,Yorkville,Sushi Restaurant,Japanese Restaurant,Fish Market,Indian Chinese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Deli / Bodega,Cocktail Bar,Grocery Store,Hawaiian Restaurant
10,Lenox Hill,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Restaurant,Bubble Tea Shop,Steakhouse,Smoothie Shop,Seafood Restaurant
13,Lincoln Square,Sushi Restaurant,Japanese Restaurant,Chinese Restaurant,Smoothie Shop,Grocery Store,Ramen Restaurant,Steakhouse,Seafood Restaurant,Sandwich Place,Sake Bar
14,Clinton,Sushi Restaurant,Japanese Restaurant,Cocktail Bar,Seafood Restaurant,Asian Restaurant,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
15,Midtown,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Seafood Restaurant,Ramen Restaurant,Bakery,Sandwich Place,Vegetarian / Vegan Restaurant,Hawaiian Restaurant,Indian Chinese Restaurant
16,Murray Hill,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Ramen Restaurant,Bakery,Restaurant,Vegetarian / Vegan Restaurant,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant
17,Chelsea,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Restaurant,Bubble Tea Shop,Steakhouse,Smoothie Shop,Seafood Restaurant
25,Manhattan Valley,Sushi Restaurant,Japanese Restaurant,Hawaiian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant
26,Morningside Heights,Sushi Restaurant,Japanese Restaurant,Hawaiian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant


In [34]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Upper West Side,Sushi Restaurant,Japanese Restaurant,Grocery Store,Asian Restaurant,Cocktail Bar,Restaurant,Bubble Tea Shop,Steakhouse,Smoothie Shop,Seafood Restaurant


In [35]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Sushi Restaurant,Japanese Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
11,Roosevelt Island,Sushi Restaurant,Asian Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
19,East Village,Sushi Restaurant,Japanese Restaurant,Fish Market,Vegetarian / Vegan Restaurant,Cocktail Bar,Restaurant,Bubble Tea Shop,Steakhouse,Smoothie Shop,Seafood Restaurant
22,Little Italy,Sushi Restaurant,Japanese Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
23,Soho,Sushi Restaurant,Japanese Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
24,West Village,Sushi Restaurant,Japanese Restaurant,Fish Market,Sake Bar,Asian Restaurant,Vegetarian / Vegan Restaurant,Deli / Bodega,Cocktail Bar,Grocery Store,Hawaiian Restaurant
28,Battery Park City,Sushi Restaurant,Japanese Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
31,Noho,Sushi Restaurant,Japanese Restaurant,Fish Market,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant
32,Civic Center,Sushi Restaurant,Japanese Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
35,Turtle Bay,Sushi Restaurant,Japanese Restaurant,Seafood Restaurant,Steakhouse,Asian Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant


In [36]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Manhattanville,Sushi Restaurant,Japanese Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
20,Lower East Side,Sushi Restaurant,Japanese Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
29,Financial District,Sushi Restaurant,Japanese Restaurant,Noodle House,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
